In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.externals import joblib
from sklearn import *
from sklearn import cluster
from sklearn.metrics import mean_squared_error,mean_absolute_error
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

/Users/emily-air13/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/emily-air13/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/emily-air13/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will 

In [2]:
k_means = joblib.load(open('kmeans.pkl', 'rb'))
loans = pd.read_csv('../loanstats.csv')
loans['kcluster'] = k_means.labels_

In [3]:
loans['term'] = loans['term'].map(lambda a: int(a.strip(' months')))
loans['application_type'] = loans['application_type'].map(lambda a: 1 if a=='Joint App' else 0)

def elength(a):
    if (a=='n/a'):
        return 0
    elif (a=='10+ years'):
        return 10
    elif (a=='1 year'):
        return 1
    elif (a=='< 1 year'):
        return 0.5
    else:
        return float(a.strip(' years'))
    
loans['emp_length'] = loans['emp_length'].map(lambda a: elength(a))

loans['revol_util'] = loans['revol_util'].map(lambda a: float(a.strip('%')))

homes = pd.get_dummies(loans['home_ownership'], prefix='home')
loans = loans.join(homes)
loans.drop('home_ownership', axis=1, inplace=True)

states = pd.get_dummies(loans['addr_state'], prefix='st')
loans = loans.join(states)
loans.drop('addr_state', axis=1, inplace=True)

loans.drop('grade', axis=1, inplace=True)
loans.drop('sub_grade', axis=1, inplace=True)
loans.drop('set', axis=1, inplace=True)
loans.drop('emp_title', axis=1, inplace=True)
loans.drop('timestamp', axis=1, inplace=True)
loans.drop('issue_d', axis=1, inplace=True)
loans.drop('last_credit_pull_d', axis=1, inplace=True)
loans.drop('title', axis=1, inplace=True)
loans.drop('purpose', axis=1, inplace=True)
loans.drop('next_pymnt_d', axis=1, inplace=True)
loans.drop('zip_code', axis=1, inplace=True)
loans.drop('last_fico_range_high', axis=1, inplace=True)
loans.drop('fico_range_high', axis=1, inplace=True)
loans.drop('last_fico_range_low', axis=1, inplace=True)
loans.drop('installment', axis=1, inplace=True)

In [4]:
kc0 = loans[loans['kcluster']==0]
kc1 = loans[loans['kcluster']==1]
kc2 = loans[loans['kcluster']==2]
kc3 = loans[loans['kcluster']==3]
kc4 = loans[loans['kcluster']==4]
kc5 = loans[loans['kcluster']==5]
kc6 = loans[loans['kcluster']==6]

kc0.drop('kcluster', axis=1, inplace=True)
kc1.drop('kcluster', axis=1, inplace=True)
kc2.drop('kcluster', axis=1, inplace=True)
kc3.drop('kcluster', axis=1, inplace=True)
kc4.drop('kcluster', axis=1, inplace=True)
kc5.drop('kcluster', axis=1, inplace=True)
kc6.drop('kcluster', axis=1, inplace=True)

/Users/emily-air13/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/emily-air13/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/emily-air13/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by I

In [5]:
array=kc0.values
Y=array[:,3]
x1=array[:,1:3]
x2=array[:,4:]
X=np.hstack((x1,x2))
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
mlp = MLPRegressor(hidden_layer_sizes=(50,20,10))
mlp.fit(X_train,Y_train)
ptrain = mlp.predict(X_train)
ptest = mlp.predict(X_test)
rms_train = sqrt(mean_squared_error(Y_train, ptrain))
rms_test = sqrt(mean_squared_error(Y_test, ptest))
mae_train = mean_absolute_error(Y_train, ptrain)
mae_test = mean_absolute_error(Y_test, ptest)
mape_train = np.mean(np.abs((Y_train - ptrain) / Y_train)) * 100
mape_test = np.mean(np.abs((Y_test - ptest) / Y_test)) * 100
#results.append([name, rms_train, rms_test, mae_train, mae_test, mape_train, mape_test])

print('|metric |train            |test             | \n|rms    |'+str(rms_train)+'|'+str(rms_test)+'|\n|mae    |'
      +str(mae_train)+'    |'+str(mae_test)+'    |\n|mape   |'+str(mape_train)+'    |'+str(mape_test)+'    |')

|metric |train            |test             | 
|rms    |3.066613709137871|3.196394696240065|
|mae    |2.34191621068    |2.44230116369    |
|mape   |19.9523024255    |20.787755507    |


In [6]:
array=kc1.values
Y=array[:,3]
x1=array[:,1:3]
x2=array[:,4:]
X=np.hstack((x1,x2))
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
mlp = MLPRegressor(hidden_layer_sizes=(50,20,10))
mlp.fit(X_train,Y_train)
ptrain = mlp.predict(X_train)
ptest = mlp.predict(X_test)
rms_train = sqrt(mean_squared_error(Y_train, ptrain))
rms_test = sqrt(mean_squared_error(Y_test, ptest))
mae_train = mean_absolute_error(Y_train, ptrain)
mae_test = mean_absolute_error(Y_test, ptest)
mape_train = np.mean(np.abs((Y_train - ptrain) / Y_train)) * 100
mape_test = np.mean(np.abs((Y_test - ptest) / Y_test)) * 100
#results.append([name, rms_train, rms_test, mae_train, mae_test, mape_train, mape_test])

print('|metric |train            |test             | \n|rms    |'+str(rms_train)+'|'+str(rms_test)+'|\n|mae    |'
      +str(mae_train)+'    |'+str(mae_test)+'    |\n|mape   |'+str(mape_train)+'    |'+str(mape_test)+'    |')

|metric |train            |test             | 
|rms    |2.7117863398528215|2.8758832440427344|
|mae    |2.07753765852    |2.20105015679    |
|mape   |16.3273114554    |17.2521632035    |


In [7]:
array=kc2.values
Y=array[:,3]
x1=array[:,1:3]
x2=array[:,4:]
X=np.hstack((x1,x2))
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
mlp = MLPRegressor(hidden_layer_sizes=(50,20,10))
mlp.fit(X_train,Y_train)
ptrain = mlp.predict(X_train)
ptest = mlp.predict(X_test)
rms_train = sqrt(mean_squared_error(Y_train, ptrain))
rms_test = sqrt(mean_squared_error(Y_test, ptest))
mae_train = mean_absolute_error(Y_train, ptrain)
mae_test = mean_absolute_error(Y_test, ptest)
mape_train = np.mean(np.abs((Y_train - ptrain) / Y_train)) * 100
mape_test = np.mean(np.abs((Y_test - ptest) / Y_test)) * 100
#results.append([name, rms_train, rms_test, mae_train, mae_test, mape_train, mape_test])

print('|metric |train            |test             | \n|rms    |'+str(rms_train)+'|'+str(rms_test)+'|\n|mae    |'
      +str(mae_train)+'    |'+str(mae_test)+'    |\n|mape   |'+str(mape_train)+'    |'+str(mape_test)+'    |')

|metric |train            |test             | 
|rms    |3.636629613762908|3.8368202773499416|
|mae    |2.64958144748    |2.7740505255    |
|mape   |20.2305102252    |21.1082463535    |


In [8]:
array=kc3.values
Y=array[:,3]
x1=array[:,1:3]
x2=array[:,4:]
X=np.hstack((x1,x2))
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
mlp = MLPRegressor(hidden_layer_sizes=(50,20,10))
mlp.fit(X_train,Y_train)
ptrain = mlp.predict(X_train)
ptest = mlp.predict(X_test)
rms_train = sqrt(mean_squared_error(Y_train, ptrain))
rms_test = sqrt(mean_squared_error(Y_test, ptest))
mae_train = mean_absolute_error(Y_train, ptrain)
mae_test = mean_absolute_error(Y_test, ptest)
mape_train = np.mean(np.abs((Y_train - ptrain) / Y_train)) * 100
mape_test = np.mean(np.abs((Y_test - ptest) / Y_test)) * 100
#results.append([name, rms_train, rms_test, mae_train, mae_test, mape_train, mape_test])

print('|metric |train            |test             | \n|rms    |'+str(rms_train)+'|'+str(rms_test)+'|\n|mae    |'
      +str(mae_train)+'    |'+str(mae_test)+'    |\n|mape   |'+str(mape_train)+'    |'+str(mape_test)+'    |')

|metric |train            |test             | 
|rms    |3.4423975892742638|3.547005841632404|
|mae    |2.61052378704    |2.68329004731    |
|mape   |20.3196472082    |20.8530544519    |


In [9]:
array=kc4.values
Y=array[:,3]
x1=array[:,1:3]
x2=array[:,4:]
X=np.hstack((x1,x2))
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
mlp = MLPRegressor(hidden_layer_sizes=(50,20,10))
mlp.fit(X_train,Y_train)
ptrain = mlp.predict(X_train)
ptest = mlp.predict(X_test)
rms_train = sqrt(mean_squared_error(Y_train, ptrain))
rms_test = sqrt(mean_squared_error(Y_test, ptest))
mae_train = mean_absolute_error(Y_train, ptrain)
mae_test = mean_absolute_error(Y_test, ptest)
mape_train = np.mean(np.abs((Y_train - ptrain) / Y_train)) * 100
mape_test = np.mean(np.abs((Y_test - ptest) / Y_test)) * 100
#results.append([name, rms_train, rms_test, mae_train, mae_test, mape_train, mape_test])

print('|metric |train            |test             | \n|rms    |'+str(rms_train)+'|'+str(rms_test)+'|\n|mae    |'
      +str(mae_train)+'    |'+str(mae_test)+'    |\n|mape   |'+str(mape_train)+'    |'+str(mape_test)+'    |')

|metric |train            |test             | 
|rms    |2.720421029031796|2.853455515994795|
|mae    |2.11265932355    |2.21630723149    |
|mape   |17.6399405192    |18.5272599744    |


In [10]:
array=kc5.values
Y=array[:,3]
x1=array[:,1:3]
x2=array[:,4:]
X=np.hstack((x1,x2))
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
mlp = MLPRegressor(hidden_layer_sizes=(50,20,10))
mlp.fit(X_train,Y_train)
ptrain = mlp.predict(X_train)
ptest = mlp.predict(X_test)
rms_train = sqrt(mean_squared_error(Y_train, ptrain))
rms_test = sqrt(mean_squared_error(Y_test, ptest))
mae_train = mean_absolute_error(Y_train, ptrain)
mae_test = mean_absolute_error(Y_test, ptest)
mape_train = np.mean(np.abs((Y_train - ptrain) / Y_train)) * 100
mape_test = np.mean(np.abs((Y_test - ptest) / Y_test)) * 100
#results.append([name, rms_train, rms_test, mae_train, mae_test, mape_train, mape_test])

print('|metric |train            |test             | \n|rms    |'+str(rms_train)+'|'+str(rms_test)+'|\n|mae    |'
      +str(mae_train)+'    |'+str(mae_test)+'    |\n|mape   |'+str(mape_train)+'    |'+str(mape_test)+'    |')

|metric |train            |test             | 
|rms    |2.4735322490107117|2.547156870842096|
|mae    |1.90541320089    |1.96188707667    |
|mape   |14.0573497203    |14.4822755711    |


In [11]:
array=kc6.values
Y=array[:,3]
x1=array[:,1:3]
x2=array[:,4:]
X=np.hstack((x1,x2))
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
mlp = MLPRegressor(hidden_layer_sizes=(50,20,10))
mlp.fit(X_train,Y_train)
ptrain = mlp.predict(X_train)
ptest = mlp.predict(X_test)
rms_train = sqrt(mean_squared_error(Y_train, ptrain))
rms_test = sqrt(mean_squared_error(Y_test, ptest))
mae_train = mean_absolute_error(Y_train, ptrain)
mae_test = mean_absolute_error(Y_test, ptest)
mape_train = np.mean(np.abs((Y_train - ptrain) / Y_train)) * 100
mape_test = np.mean(np.abs((Y_test - ptest) / Y_test)) * 100
#results.append([name, rms_train, rms_test, mae_train, mae_test, mape_train, mape_test])

print('|metric |train            |test             | \n|rms    |'+str(rms_train)+'|'+str(rms_test)+'|\n|mae    |'
      +str(mae_train)+'    |'+str(mae_test)+'    |\n|mape   |'+str(mape_train)+'    |'+str(mape_test)+'    |')

|metric |train            |test             | 
|rms    |2.6080194642200327|2.7286924694218615|
|mae    |2.01808081968    |2.11403875831    |
|mape   |17.2246311117    |18.0484509125    |
